In [2]:
import os
import random
import tensorflow as tf

from pathlib import Path
from tensorflow.keras.applications import resnet50
from tensorflow.keras import (Sequential, preprocessing, layers, optimizers, Model)

In [3]:
# setting the paths
DATA_DIRECTORY = Path('/content/drive/MyDrive/data/flowers_dataset')
TRAIN_DATA = DATA_DIRECTORY / "train"
TEST_DATA = DATA_DIRECTORY / "test"

# setting hyperparameters
IMAGE_SIZE = 150
BATCH_SIZE = 32
TRAIN_RATIO = 0.8
LR_RATE = 1e-3
EPOCHS = 10

In [4]:
def parse_image(filename):
    """
    Takes in the filename, reads, decodes, resizes and returns the image.
    """
    
    image = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [IMAGE_SIZE, IMAGE_SIZE])
    return image

In [5]:
def _datasets(data_directory):
    """
    Takes in the data directory with class directories and return the tf datasets with number of classes.
    """
    
    # getting all the class names
    classes = os.listdir(data_directory)
    
    if ".DS_Store" in classes:
        classes.remove(".DS_Store")

    # fetching all the image filenames
    filenames = list(data_directory.glob("**/*.jpg"))
    random.shuffle(filenames)

    # creating and one-hot encoding the target labels
    labels = [classes.index(str(name).split("/")[-2]) for name in filenames]
    num_classes = max(labels) + 1
    labels_onehot = tf.one_hot(labels, num_classes)

    filenames = [str(name) for name in filenames]

    # creating the tf dataset from the filenames
    files_dataset = tf.data.Dataset.from_tensor_slices(filenames)

    # getting the image count
    image_count = files_dataset.cardinality().numpy()
    
    # reading the images
    images_dataset = files_dataset.map(parse_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)

    # creating the tf dataset for the labels
    labels_dataset = tf.data.Dataset.from_tensor_slices(labels_onehot)

    # combining the images, labels and shuffling
    dataset = tf.data.Dataset.zip((images_dataset, labels_dataset))
    dataset = dataset.shuffle(buffer_size=50)

    # splitting the data into train and validation set
    train_dataset = dataset.take(image_count * TRAIN_RATIO)
    validation_dataset = dataset.skip(image_count * TRAIN_RATIO)

    # batching and prefetching
    train_dataset = train_dataset.batch(BATCH_SIZE)
    train_dataset = train_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

    validation_dataset = validation_dataset.batch(BATCH_SIZE)
    validation_dataset = validation_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

    return train_dataset, validation_dataset, num_classes

In [37]:
def _model(num_classes):
    """
    Takes in the number of classes and creates the CNN model.
    """

    # creating a mini-model for data-augmentation
    data_augmentation = Sequential([
        layers.RandomFlip("horizontal", input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3), seed=32),
        layers.RandomRotation(factor=(-0.1, 0.1), seed=42),
        layers.RandomZoom(height_factor=(-0.1, 0.1), width_factor=(-0.1, 0.1), seed=42),
        layers.RandomTranslation(height_factor=(-0.05, 0.05), width_factor=(-0.05, 0.05), seed=32)
    ])

    # base_model = resnet50.ResNet50(
    #     weights="imagenet",
    #     include_top=False
    # )

    image_input = layers.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3), name="image")
    x = data_augmentation(image_input)

    # x = resnet50.preprocess_input(x)
    # x = base_model(x, training=False)

    x = layers.Conv2D(32, (5,5), activation='relu')(x)
    x = layers.MaxPooling2D((2,2))(x)
    x = layers.Conv2D(64, (3,3), activation='relu')(x)
    x = layers.MaxPooling2D((2,2), (2,2))(x)
    x = layers.Conv2D(96, (3,3), activation='relu')(x)
    x = layers.MaxPooling2D((2,2), (2,2))(x)
    x = layers.Conv2D(96, (3,3), activation='relu')(x)
    x = layers.MaxPooling2D((2,2), (2,2))(x)
    x = layers.Flatten()(x)
    x = layers.Dense(512, activation="relu")(x)
    
    outputs = layers.Dense(num_classes, activation="softmax")(x)

    model = Model(inputs=image_input, outputs=outputs)

    print(model.summary())

    return model

In [38]:
# fetching the datasets
train_set, val_set, num_classes = _datasets(TRAIN_DATA)

In [39]:
model = _model(num_classes)

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 image (InputLayer)          [(None, 150, 150, 3)]     0         
                                                                 
 sequential_9 (Sequential)   (None, 150, 150, 3)       0         
                                                                 
 conv2d_25 (Conv2D)          (None, 146, 146, 32)      2432      
                                                                 
 max_pooling2d_24 (MaxPoolin  (None, 73, 73, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_26 (Conv2D)          (None, 71, 71, 64)        18496     
                                                                 
 max_pooling2d_25 (MaxPoolin  (None, 35, 35, 64)       0         
 g2D)                                                      

In [40]:
model.compile(optimizer=optimizers.Adam(learning_rate=LR_RATE), 
            loss="categorical_crossentropy",
            metrics=["accuracy"]
)

In [41]:
model.fit(train_set, validation_data=val_set, epochs = 25, verbose=1)

Epoch 1/25
68/68 [==============================] - 17s 218ms/step - loss: 7.8746 - accuracy: 0.2891 - val_loss: 1.4916 - val_accuracy: 0.3284
Epoch 2/25
68/68 [==============================] - 15s 218ms/step - loss: 1.4527 - accuracy: 0.3585 - val_loss: 1.4916 - val_accuracy: 0.3541
Epoch 3/25
68/68 [==============================] - 15s 219ms/step - loss: 1.3732 - accuracy: 0.4044 - val_loss: 1.6241 - val_accuracy: 0.3450
Epoch 4/25
68/68 [==============================] - 15s 221ms/step - loss: 1.2363 - accuracy: 0.4802 - val_loss: 1.4160 - val_accuracy: 0.4055
Epoch 5/25
68/68 [==============================] - 15s 218ms/step - loss: 1.1526 - accuracy: 0.5290 - val_loss: 1.0519 - val_accuracy: 0.5670
Epoch 6/25
68/68 [==============================] - 15s 219ms/step - loss: 1.0558 - accuracy: 0.5740 - val_loss: 1.0535 - val_accuracy: 0.5615
Epoch 7/25
68/68 [==============================] - 15s 221ms/step - loss: 1.0131 - accuracy: 0.5864 - val_loss: 1.1588 - val_accuracy: 0.5193